# IConfucius 提示设计用于 llama_cpp_canister

## 中文版本

# 设置

## 验证我们是否在 Conda 环境中

In [ ]:
import sys

print(sys.executable)

## 导入 Python 包

In [ ]:
import os
import sys
import json
import base64
import io

# from dotenv import load_dotenv
# import requests
import pprint
from pathlib import Path
import subprocess
import jupyter_black
import textwrap

import random

from run_llama_cpp import run_llama_cpp

# Activate the jupyter_black extension, which reformats code cells with black
# https://github.com/n8henrie/jupyter-black
jupyter_black.load()

In [ ]:
# ################################################################
# 先构建常规的 llama.cpp，然后指定其路径 
# 
# 要构建 llama.cpp，请按照 README 中的说明进行操作：
# https://github.com/ggml-org/llama.cpp  
#    
# 定义 llama-cli 的位置，相对于此笔记本的路径  
# ################################################################


# 使用最新版本的 llama.cpp
# LLAMA_CLI_PATH = "../../ggml_org_llama_latest.cpp/build/bin/llama-cli"

# 当前版本的 llama_cpp_canister 使用的是 llama.cpp Git SHA 615212
LLAMA_CLI_PATH = "../../ggml_org_llama_615212.cpp/build/bin/llama-cli"


# #######################################################################
# 选择 MODEL_TYPE 和 MODEL（位置相对于此笔记本）
# #######################################################################

seed = random.randint(0, 10000000)
num_tokens = 1024
temp = 0.7
# top_k = 50
# top_p = 0.95
# min_p = 0.05
# tfs = 0.9
# typical = 0.9
# mirostat = 2
# mirostat_lr = 0.1
# mirostat_ent = 5.0
repeat_penalty = 1.1

# Notes:
#                                     <not quantized>|<         quantized                >
#  --cache-type-k has allowed values: f32, f16, bf16, q8_0, q4_0, q4_1, iq4_nl, q5_0, q5_1
#  --cache-type-v is not tested because that requires a GPU,
#                 which is not available right now in an Internet Computer canister

# ------------------------------------------------------------------------------------------
# 163 Million parameters

# https://huggingface.co/tensorblock/gpt2-GGUF (124M)
# MODEL_TYPE = "gpt2"
# MODEL = "../llms/llama_cpp_canister/models/tensorblock/gpt2-GGUF/gpt2-Q8_0.gguf"
# cache_type_k = "f16"

# ------------------------------------------------------------------------------------------
# 630 Million parameters

# https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct-GGUF
# MODEL_TYPE = "Qwen"
# MODEL = "../llms/llama_cpp_canister/models/Qwen/Qwen2.5-0.5B-Instruct-GGUF/qwen2.5-0.5b-instruct-q4_k_m.gguf"
# cache_type_k = "f16"

MODEL_TYPE = "Qwen"
MODEL = "../llms/llama_cpp_canister/models/Qwen/Qwen2.5-0.5B-Instruct-GGUF/qwen2.5-0.5b-instruct-q8_0.gguf"
cache_type_k = "q8_0"


# ------------------------------------------------------------------------------------------
# 1.24 Billion parameters

# https://huggingface.co/unsloth/Llama-3.2-1B-Instruct-GGUF
# MODEL_TYPE = "Llama-3.2"
# MODEL = "../llms/llama_cpp_canister/models/unsloth/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct-Q4_K_M.gguf"
# cache_type_k = "q5_0"

# ------------------------------------------------------------------------------------------

prompt = ""
topic = "加密货币"
# topic = "Bitcoin"
system_prompt = "你是孔子，古代的哲学家。你以深刻且富有同情心的方式结束你的名言。"
user_prompt = f"写一句深刻且发人深省的名言关于 {topic}。 只提供名言，其他不作提供。"

if MODEL_TYPE in ["SmolLM2", "Qwen"]:
    prompt = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n<|im_start|>assistant\n"
elif MODEL_TYPE == "Llama-3.2":
    prompt = f"<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
else:
    print(f"Model type {MODEL_TYPE} not recognized")
    exit(1)

print("\nprompt:\n", textwrap.fill(prompt, width=80))

run_llama_cpp(
    LLAMA_CLI_PATH,
    MODEL,
    prompt,
    num_tokens,
    seed,
    temp,
    # top_k,
    # top_p,
    # min_p,
    # tfs,
    # typical,
    # mirostat,
    # mirostat_lr,
    # mirostat_ent,
    repeat_penalty,
    cache_type_k,
)